**Training noteebook can found here https://www.kaggle.com/razatabish/ranzr-clip**

In [ ]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

In [ ]:
import numpy as np
import pandas as pd 
import os
import time 
import albumentations as A
from albumentations.pytorch import ToTensorV2
from collections import defaultdict
from tqdm import tqdm
from sklearn import model_selection, preprocessing 
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import copy

import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from torch.utils.data import Dataset, DataLoader 
import torchvision
from torchvision import models, transforms 
from torch.cuda.amp import autocast, GradScaler


In [ ]:
params = {
    "model": "efficientnet_b3", #"model": "resnet50",
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "lr": 0.0001,
    "batch_size": 4,
    "num_workers": 16,
    "num_epochs": 10,
    "T_0":6, # CosineAnnealingWarmRestarts
    "min_lr":1e-6
}

In [ ]:
ROOT_DIR = "../input/ranzcr-clip-catheter-line-classification"
test_dir = "../input/ranzcr-clip-catheter-line-classification/test/"
model_path = "../input/ranzr-clip-train-pytorch/"

In [ ]:
os.listdir(ROOT_DIR)

In [ ]:
df = pd.read_csv(os.path.join(ROOT_DIR, "sample_submission.csv"))

In [ ]:
df.head(5)

In [ ]:
len(df['StudyInstanceUID'].unique())

In [ ]:
classes = df.columns[1:]

In [ ]:
len(classes)

In [ ]:
labels = df[classes]

In [ ]:
labels

In [ ]:
#img_list = os.list_dir(os.path.join(ROOT_DIR, df["StudyInstanceUID"]+".jpg"))

In [ ]:
classes = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal',
       'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal',
       'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
       'Swan Ganz Catheter Present']

In [ ]:
class RANZRDataset(Dataset):
    def __init__(self, data_dir, df, transform=None):
        self.data_dir = data_dir
        self.df = df
        self.files = df["StudyInstanceUID"].values
        self.transform = transform
        
    def __getitem__(self, idx):
        img_name = self.files[idx]
        file_path = f"{self.data_dir}{img_name}"+".jpg"
        image = Image.open(file_path).convert('RGB')
        image = np.array(image)
#         image = cv2.imread(file_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        #print(file_path)
        if self.transform:
            augmented = self.transform(image = image)
            image = augmented["image"]
        
    
        
        return image
    def __len__(self):
        return len(self.df)
        
        

In [ ]:
dataset = RANZRDataset(test_dir, df)
img = dataset[0]
plt.imshow(img)
plt.show()
img.shape

In [ ]:
def get_valid_transform():
    return A.Compose([
        A.RandomResizedCrop(300, 300),
        #A.Flip(0.5),
        #A.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
        ToTensorV2(),

    ])

In [ ]:
# import timm

# class RANZRModel(nn.Module):
#     def __init__(self, model_name=params["model"], pretrained=False):
#         super().__init__()
#         self.model = timm.create_model(model_name, pretrained=pretrained)
#         n_features = self.model.fc.in_features
#         self.model.fc = nn.Linear(n_features, len(classes))

#     def forward(self, x):
#         x = self.model(x)
#         return x
#efficientnet
import timm
#EFFICIENTNET-B7
class RANZRModel(nn.Module):
    def __init__(self, model_name=params["model"], pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, len(classes))

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def inference(model,states, test_loader):
    preds = []
    model.to("cpu")
    for image in tqdm(test_loader, total = len(test_loader)):
        avg_preds = []
        image = image.to("cpu").float()
        for state in states:
            model.load_state_dict(state)
            model.eval() 
            with torch.no_grad():
                y_preds = model(image)
                avg_preds.append(y_preds.sigmoid().to('cpu').numpy())
        avg_preds = np.mean(avg_preds, axis=0)
        preds.append(avg_preds)

    preds = np.concatenate(preds)
    return preds

In [ ]:
models_weights = []
for file in os.listdir(model_path):
    if file.endswith(".pth"):
        models_weights.append(file)
models_weights

In [ ]:
model = RANZRModel(model_name=params["model"], pretrained=False)
model.to("cpu")
states = [torch.load(f"{model_path}{weights}", map_location=torch.device('cpu')) for weights in models_weights] 
#states
test_dataset = RANZRDataset(test_dir, df, get_valid_transform())
test_loader = DataLoader(test_dataset, shuffle=False, pin_memory = True)
predictions = inference(model, states, test_loader)

df[classes] = predictions
df[['StudyInstanceUID'] + classes].to_csv('./submission.csv', index=False)
df.head()
